In [ ]:
from utils import get_model
import torch
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
from torchvision.datasets import MNIST
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader
from svhn import SVHN
from mnist_m import MNIST_M
import sys , os

import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

os.environ["CUDA_VISIBLE_DEVICES"]= "7" 

In [ ]:
model = get_model("wrn-40-2", num_classes=10)
model = torch.nn.DataParallel(model).cuda()
model.load_state_dict(torch.load("SVHN_only/checkpoint-epoch70.pt")["state_dict"]);
model.eval()

transform_list = [transforms.ToTensor()]
transform_train = transforms.Compose(transform_list)

# dataset = MNIST_M("/newfoundland/tarun/datasets/Digits/mnist_m/" , split="train" , transform=transform_train)
dataset = SVHN(root="/newfoundland/tarun/datasets/Digits/SVHN/" , split="extra_svhn" , transform=transform_train)
dataloader = DataLoader(dataset , batch_size=512 , shuffle=False)

In [ ]:
pseudo_labels = []
gt_labels = []

for i , (image , label) in enumerate(dataloader):
    
    sys.stdout.write(f"\r{i+1}/{len(dataloader)}")
    sys.stdout.flush()
        
    image = image.cuda()
#     label = label.cuda()
#     
    logits = model(image)
    predictions = logits.argmax(1)
    pseudo_labels.extend(predictions.data.cpu().numpy())
    gt_labels.extend(label.cpu().numpy())
    
#     plt.imshow(image.cpu().numpy()[0].transpose(2,1,0))
#     plt.show()
#     print("Label" , label[0])
#     print("Predicted" , logits[0].argmax()) 

In [ ]:
sum([1 for i in range(len(gt_labels)) if pseudo_labels[i] == gt_labels[i]])/len(pseudo_labels)*100

In [ ]:
sum([1 for i in range(len(gt_labels)) if pseudo_labels[i] == gt_labels[i]])/len(pseudo_labels)*100

In [2]:
import numpy as np
# svhn_to_svhn_pseudo = np.array(pseudo_labels).reshape(-1,1)

In [ ]:
np.save("svhn_to_svhn_pseudo.np" , svhn_to_svhn_pseudo)

In [3]:
labels = np.load("svhn_to_svhn_pseudo.np.npy")
gt_labels = np.array(gt_labels).reshape(-1,1)
(gt_labels == labels).sum()/len(gt_labels)

NameError: name 'gt_labels' is not defined

In [4]:
labels["y"].shape

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
pseudo_labels[-10:]

In [ ]:
np.save("gt_labels_svhn_extra" , gt_labels)